In [ ]:
$pip install pmdarima

In [23]:
import os
import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader



class CustomDataset(Dataset):
    # 반드시 init, len, getitem 구현
    def __init__(self, df, n_past, n_future):
        # 빈 리스트 생성 <- 데이터 저장
        self.X = []     # n_past 만큼의 feature 데이터
        self.y = []     # n_future 만큼의 label 데이터
        x_col = (df.shape[1]) - 1   # df 에서 -1번째 columns 까지 x
        for i in range(n_past, len(df) - n_future + 1): # -1 말고 변수로
            self.X.append(df[i - n_past:i, 0:x_col])
            self.y.append(df[i + n_future - 1: i + n_future, x_col])

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# TODO: argparse 바꾸기

# Device Setting
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Initializing Device: {device}')

# Data Setting
file_path = os.path.join(os.getcwd(), "data/01000002.txt")
total_data = pd.read_csv(file_path, sep="\t", index_col="Timestamp")
# total_data = total_data.loc[:, ["AccZ", "Str1", "Str2", "Str3"]]
total_data = total_data.loc[:, ["Str1", "Str2", "Str3"]]

minmax_scaler = MinMaxScaler(feature_range=(-1, 1))
minmax_scaler = minmax_scaler.fit(total_data)
total_data_scaled = minmax_scaler.transform(total_data)

minmax_scaler2 = minmax_scaler.fit(np.array(total_data.Str3[:]).reshape(-1, 1))
total_data_scaled2 = minmax_scaler2.transform(np.array(total_data.Str3[:]).reshape(-1, 1))


train_valid_split = int(len(total_data_scaled) * 0.3)   #argparse
df_train = total_data_scaled[:-train_valid_split]
df_valid = total_data_scaled[-train_valid_split:]
df_train.shape, df_valid.shape
# print(df_train.shape, df_valid.shape)

# train_data = CustomDataset(df_train, 30, 1)     #argparse
# valid_data = CustomDataset(df_valid, 30, 1)

# batch_size = 100     #argparse
# train_loader = DataLoader(train_data, batch_size=batch_size, drop_last=True, num_workers=8)    # drop_last = drop the last incomplete batch
# valid_loader = DataLoader(valid_data, batch_size=batch_size, drop_last=True, num_workers=8)
X = df_train[:, :2]
y = df_train[:, 2]
X.shape, y.shape

Initializing Device: cpu


((21000, 2), (21000,))

In [24]:
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA

model_arima = auto_arima(y, trace=True, error_action='ignore', start_p=1,
                        start_q=1, max_p=3, max_q=3, suppress_warnings=True,
                        stepwise=False, seasonal=False)

model_arima.fit(y)

 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-55601.306, Time=2.21 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=10.83 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-101589.845, Time=25.90 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=-115744.110, Time=27.95 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-67112.317, Time=1.47 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=14.29 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-108616.186, Time=20.70 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-103300.316, Time=26.72 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-102667.136, Time=2.47 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-119699.770, Time=24.23 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-125246.343, Time=26.05 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-126728.084, Time=32.39 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-105769.510, Time=2.37 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-120221.436, Time=25.87 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=

ARIMA(order=(2, 1, 3), scoring_args={}, suppress_warnings=True)

In [21]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(y, order=(2, 1, 3))
model_fit = model.fit(disp=0)
print(model_fit.summary())
model_fit.plot_predict(dynamic=False)

plt.show()

NotImplementedError: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been removed in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and statsmodels.tsa.SARIMAX.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained. It also offers alternative specialized
parameter estimators.


In [19]:
model = ARIMA(y, order=(2, 1, 3))  
fitted = model.fit(disp=-1)  

# Forecast
fc, se, conf = fitted.forecast(15, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc)#, index=[79,80,81,82,83,84,85,86,87,88,89,90,91,92,93])

# Plot
plt.figure(figsize=(12, 5), dpi=100)
plt.plot(y, label='solid ticket')
plt.plot(fc_series, label='forecast')
plt.title('Forecast')
plt.legend(loc='upper right', fontsize=8)
plt.show()

NotImplementedError: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been removed in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and statsmodels.tsa.SARIMAX.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained. It also offers alternative specialized
parameter estimators.
